In [1]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression,Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor,HistGradientBoostingRegressor, AdaBoostRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from transformation_functions import *
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


In [2]:
X=import_data() #import dataframe
X=X.drop(['atemp','hum','weekday'],axis=1) #drop columns due to multicollinearity 
X=transform_feature_weathersit(X) #replace value 4 with value 3 for feature 'weathersit'
X=create_lagged_values(X) # create lag values for features 'temp','cnt', 'windspeed', 'weathersit'

#Separate features and target variable
Features_set=X.drop(['cnt'],axis=1)
Target_set=X['cnt']

#normalize 'cnt' values

X_train, X_test, y_train, y_test = train_test_split(Features_set, Target_set, test_size=0.33, random_state=42)

#normalize cnt values
scaler = StandardScaler()
y_train=scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test=scaler.transform(y_test.values.reshape(-1, 1))
X_train,y_train=normalize_lag_values(scaler,X_train,y_train) #normalize lag values of cnt features using .transform
X_test,y_test=normalize_lag_values(scaler,X_test,y_test) #normalize lag values of cnt features using .transform

C:\Users\user\Untitled Folder\transformation_functions.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['cnt'] = bike_sharing_dataset.data.targets


In [3]:
categorical_cols = ['season','mnth','hr','lagged_weathersit_25h','lagged_weathersit_24h']


# Instantiate the StandardScaler

# Instantiate the OneHotEncoder
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False) 

categorical_cols=['lagged_weathersit_24h','lagged_weathersit_25h','season', 'mnth', 'hr']


# Create ColumnTransformer 
column_transformer = ColumnTransformer(
    transformers=[('onehot', one_hot_encoder, categorical_cols)],
    remainder='passthrough'  # Pass through the remaining (numerical) columns
)





# Fit and transform the data
encoded_data_train = column_transformer.fit_transform(X_train)
encoded_data_test= column_transformer.transform(X_test)
# convert back to a DataFrame
encoded_data_train_df = pd.DataFrame(
    encoded_data_train, 
    columns=column_transformer.get_feature_names_out()  # Get the new column names
)

encoded_data_test_df = pd.DataFrame(
    encoded_data_test, 
    columns=column_transformer.get_feature_names_out()  # Get the new column names
)
encoded_data_test_df

X_train, X_test=drop_columns(encoded_data_train_df, encoded_data_test_df) #drop not relevant columns

In [4]:
X_test[['remainder__lagged_count_7d1h']].isnull().sum().sum()


0

In [5]:
#lets try firstly some algorithms without fine tuning in order to decide which one to use afterwards

# Define a dictionary to store results
results = {}

# Run different regression models
models = [
    ('Linear Regression', LinearRegression()),
    ('Ridge Regression', Ridge(alpha=1)),
    ('Lasso Regression', Lasso(alpha=0.1)),
    ('RandomForestRegressor',RandomForestRegressor()),
    ('HistGradientBoostingRegressor',HistGradientBoostingRegressor()),
    ('AdaBoostRegressor',AdaBoostRegressor()),
    ('CatBoostRegressor',CatBoostRegressor())
]

for name, model in models:
    # Fit the model
    model.fit(X_train, y_train)

    # Predict on train and test sets
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Calculate MAE for train and test sets
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mae_test = mean_absolute_error(y_test, y_pred_test)

    # Store results in the dictionary
    results[name] = {'Train MAE': mae_train, 'Test MAE': mae_test}

# Print the results in a table
print('Mean Absolute Error (MAE) Results:')
print('{:25s} {:15s} {:15s}'.format('Model Name', 'Train MAE', 'Test MAE'))
for name, metrics in results.items():
    print('{:25s} {:15.2f} {:15.2f}'.format(name, metrics['Train MAE'], metrics['Test MAE']))


Learning rate set to 0.06025
0:	learn: 0.9595791	total: 144ms	remaining: 2m 24s
1:	learn: 0.9196419	total: 147ms	remaining: 1m 13s
2:	learn: 0.8826806	total: 150ms	remaining: 49.8s
3:	learn: 0.8487828	total: 152ms	remaining: 37.9s
4:	learn: 0.8168598	total: 155ms	remaining: 30.8s
5:	learn: 0.7878206	total: 157ms	remaining: 26.1s
6:	learn: 0.7617504	total: 160ms	remaining: 22.8s
7:	learn: 0.7367367	total: 163ms	remaining: 20.3s
8:	learn: 0.7130638	total: 166ms	remaining: 18.3s
9:	learn: 0.6916738	total: 169ms	remaining: 16.7s
10:	learn: 0.6707631	total: 171ms	remaining: 15.4s
11:	learn: 0.6527926	total: 174ms	remaining: 14.3s
12:	learn: 0.6364679	total: 177ms	remaining: 13.5s
13:	learn: 0.6215277	total: 180ms	remaining: 12.7s
14:	learn: 0.6071596	total: 183ms	remaining: 12s
15:	learn: 0.5946067	total: 186ms	remaining: 11.5s
16:	learn: 0.5811147	total: 190ms	remaining: 11s
17:	learn: 0.5703757	total: 193ms	remaining: 10.5s
18:	learn: 0.5604295	total: 196ms	remaining: 10.1s
19:	learn: 0.5

237:	learn: 0.3474890	total: 715ms	remaining: 2.29s
238:	learn: 0.3471095	total: 718ms	remaining: 2.28s
239:	learn: 0.3468472	total: 720ms	remaining: 2.28s
240:	learn: 0.3464917	total: 722ms	remaining: 2.27s
241:	learn: 0.3463340	total: 725ms	remaining: 2.27s
242:	learn: 0.3460838	total: 727ms	remaining: 2.27s
243:	learn: 0.3458726	total: 730ms	remaining: 2.26s
244:	learn: 0.3454696	total: 733ms	remaining: 2.26s
245:	learn: 0.3452185	total: 735ms	remaining: 2.25s
246:	learn: 0.3449865	total: 738ms	remaining: 2.25s
247:	learn: 0.3447274	total: 741ms	remaining: 2.25s
248:	learn: 0.3443596	total: 743ms	remaining: 2.24s
249:	learn: 0.3439790	total: 745ms	remaining: 2.23s
250:	learn: 0.3438177	total: 748ms	remaining: 2.23s
251:	learn: 0.3435019	total: 750ms	remaining: 2.23s
252:	learn: 0.3431683	total: 753ms	remaining: 2.22s
253:	learn: 0.3429822	total: 755ms	remaining: 2.22s
254:	learn: 0.3427912	total: 757ms	remaining: 2.21s
255:	learn: 0.3423626	total: 760ms	remaining: 2.21s
256:	learn: 

425:	learn: 0.3090615	total: 1.25s	remaining: 1.69s
426:	learn: 0.3088274	total: 1.25s	remaining: 1.68s
427:	learn: 0.3086757	total: 1.26s	remaining: 1.68s
428:	learn: 0.3086035	total: 1.26s	remaining: 1.68s
429:	learn: 0.3084962	total: 1.26s	remaining: 1.68s
430:	learn: 0.3084183	total: 1.27s	remaining: 1.67s
431:	learn: 0.3082256	total: 1.27s	remaining: 1.67s
432:	learn: 0.3080257	total: 1.27s	remaining: 1.67s
433:	learn: 0.3077872	total: 1.27s	remaining: 1.66s
434:	learn: 0.3076013	total: 1.28s	remaining: 1.66s
435:	learn: 0.3074559	total: 1.28s	remaining: 1.66s
436:	learn: 0.3073516	total: 1.28s	remaining: 1.66s
437:	learn: 0.3072050	total: 1.29s	remaining: 1.65s
438:	learn: 0.3069926	total: 1.29s	remaining: 1.65s
439:	learn: 0.3068012	total: 1.29s	remaining: 1.65s
440:	learn: 0.3065828	total: 1.3s	remaining: 1.65s
441:	learn: 0.3064426	total: 1.3s	remaining: 1.64s
442:	learn: 0.3062735	total: 1.3s	remaining: 1.64s
443:	learn: 0.3060510	total: 1.31s	remaining: 1.64s
444:	learn: 0.3

636:	learn: 0.2809258	total: 1.8s	remaining: 1.02s
637:	learn: 0.2808104	total: 1.8s	remaining: 1.02s
638:	learn: 0.2806601	total: 1.8s	remaining: 1.02s
639:	learn: 0.2805217	total: 1.8s	remaining: 1.01s
640:	learn: 0.2804360	total: 1.81s	remaining: 1.01s
641:	learn: 0.2802982	total: 1.81s	remaining: 1.01s
642:	learn: 0.2801599	total: 1.81s	remaining: 1.01s
643:	learn: 0.2799896	total: 1.82s	remaining: 1s
644:	learn: 0.2798197	total: 1.82s	remaining: 1s
645:	learn: 0.2797652	total: 1.82s	remaining: 998ms
646:	learn: 0.2796901	total: 1.82s	remaining: 995ms
647:	learn: 0.2796108	total: 1.83s	remaining: 992ms
648:	learn: 0.2794292	total: 1.83s	remaining: 989ms
649:	learn: 0.2793296	total: 1.83s	remaining: 986ms
650:	learn: 0.2791539	total: 1.83s	remaining: 983ms
651:	learn: 0.2790371	total: 1.84s	remaining: 980ms
652:	learn: 0.2788998	total: 1.84s	remaining: 977ms
653:	learn: 0.2788388	total: 1.84s	remaining: 974ms
654:	learn: 0.2787505	total: 1.84s	remaining: 971ms
655:	learn: 0.2786474	

828:	learn: 0.2615797	total: 2.32s	remaining: 479ms
829:	learn: 0.2614593	total: 2.33s	remaining: 477ms
830:	learn: 0.2613386	total: 2.33s	remaining: 474ms
831:	learn: 0.2612066	total: 2.33s	remaining: 471ms
832:	learn: 0.2611164	total: 2.33s	remaining: 468ms
833:	learn: 0.2610100	total: 2.34s	remaining: 465ms
834:	learn: 0.2608883	total: 2.34s	remaining: 462ms
835:	learn: 0.2608049	total: 2.34s	remaining: 460ms
836:	learn: 0.2607655	total: 2.35s	remaining: 457ms
837:	learn: 0.2607017	total: 2.35s	remaining: 454ms
838:	learn: 0.2606023	total: 2.35s	remaining: 451ms
839:	learn: 0.2605432	total: 2.35s	remaining: 448ms
840:	learn: 0.2603852	total: 2.35s	remaining: 445ms
841:	learn: 0.2602449	total: 2.36s	remaining: 442ms
842:	learn: 0.2602012	total: 2.36s	remaining: 440ms
843:	learn: 0.2600953	total: 2.36s	remaining: 437ms
844:	learn: 0.2599994	total: 2.37s	remaining: 434ms
845:	learn: 0.2598700	total: 2.37s	remaining: 431ms
846:	learn: 0.2597779	total: 2.37s	remaining: 428ms
847:	learn: 

In [26]:

rf_param_dist = {
    "n_estimators": range(1, 200, 50),
    "max_depth": range(5,15),
    "min_samples_split": range(2, 20),
    "max_features": ["auto", "sqrt", "log2"]  # Categorical distribution
}

# Define hyperparameter distributions for HistGradientBoostingRegressor
hgb_param_dist = {
    "learning_rate": [0.01, 0.1,0.5, 1],
    "max_depth": range(5, 15),
    "l2_regularization": [0.01, 1, 10],
    "min_samples_leaf": range(2, 20,2)
}

catboost_grid = {
   "learning_rate": [0.01, 0.1, 0.5],
   "depth": [4, 6, 8, 10],
   "l2_leaf_reg": [1, 3, 5, 7],
    "iterations":[100]}



# Perform hyperparameter tuning using RandomizedSearchCV
rf_model = RandomForestRegressor()
rf_cv = RandomizedSearchCV(rf_model, rf_param_dist, cv=5, n_iter=50, scoring="neg_mean_squared_error")
rf_cv.fit(X_train, y_train)

hgb_model = HistGradientBoostingRegressor()
hgb_cv = RandomizedSearchCV(hgb_model, hgb_param_dist, cv=5, n_iter=50, scoring="neg_mean_squared_error")
hgb_cv.fit(X_train, y_train)
    
cb_model = CatBoostRegressor()
cb_cv = RandomizedSearchCV(cb_model, catboost_grid, cv=5, n_iter=50, scoring="neg_mean_squared_error")
cb_cv.fit(X_train, y_train)

# Print the best hyperparameters for each model
print("Best RandomForestRegressor hyperparameters:", rf_cv.best_params_)
print("Best HistGradientBoostingRegressor hyperparameters:", hgb_cv.best_params_)
print("Best CatBoost hyperparameters:", cb_cv.best_params_)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 48 is smaller than n_iter=50. Running 48 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


0:	learn: 0.9922762	total: 2.06ms	remaining: 204ms
1:	learn: 0.9859200	total: 4.02ms	remaining: 197ms
2:	learn: 0.9792523	total: 5.81ms	remaining: 188ms
3:	learn: 0.9728185	total: 7.6ms	remaining: 182ms
4:	learn: 0.9662532	total: 9.31ms	remaining: 177ms
5:	learn: 0.9600053	total: 11.1ms	remaining: 174ms
6:	learn: 0.9540574	total: 12.9ms	remaining: 172ms
7:	learn: 0.9478858	total: 14.7ms	remaining: 169ms
8:	learn: 0.9418815	total: 16.6ms	remaining: 168ms
9:	learn: 0.9359571	total: 18.4ms	remaining: 166ms
10:	learn: 0.9301359	total: 20.2ms	remaining: 163ms
11:	learn: 0.9240594	total: 21.9ms	remaining: 161ms
12:	learn: 0.9180855	total: 23.6ms	remaining: 158ms
13:	learn: 0.9121296	total: 25.3ms	remaining: 156ms
14:	learn: 0.9063962	total: 26.9ms	remaining: 152ms
15:	learn: 0.9007493	total: 28.5ms	remaining: 150ms
16:	learn: 0.8950999	total: 30.1ms	remaining: 147ms
17:	learn: 0.8896357	total: 31.7ms	remaining: 144ms
18:	learn: 0.8839287	total: 33.2ms	remaining: 142ms
19:	learn: 0.8785844	to

0:	learn: 0.9944904	total: 1.91ms	remaining: 190ms
1:	learn: 0.9879440	total: 3.68ms	remaining: 180ms
2:	learn: 0.9815020	total: 5.6ms	remaining: 181ms
3:	learn: 0.9751193	total: 7.02ms	remaining: 168ms
4:	learn: 0.9685163	total: 8.5ms	remaining: 162ms
5:	learn: 0.9622721	total: 10.2ms	remaining: 159ms
6:	learn: 0.9562642	total: 11.5ms	remaining: 153ms
7:	learn: 0.9502761	total: 13ms	remaining: 150ms
8:	learn: 0.9442590	total: 14.6ms	remaining: 147ms
9:	learn: 0.9383362	total: 16ms	remaining: 144ms
10:	learn: 0.9324450	total: 17.6ms	remaining: 143ms
11:	learn: 0.9263801	total: 19.1ms	remaining: 140ms
12:	learn: 0.9203931	total: 20.8ms	remaining: 139ms
13:	learn: 0.9145301	total: 22.4ms	remaining: 138ms
14:	learn: 0.9087726	total: 24.1ms	remaining: 137ms
15:	learn: 0.9031140	total: 25.7ms	remaining: 135ms
16:	learn: 0.8975238	total: 27.4ms	remaining: 134ms
17:	learn: 0.8920027	total: 29ms	remaining: 132ms
18:	learn: 0.8863394	total: 30.7ms	remaining: 131ms
19:	learn: 0.8810120	total: 32

0:	learn: 0.9948140	total: 1.63ms	remaining: 162ms
1:	learn: 0.9882184	total: 3.15ms	remaining: 154ms
2:	learn: 0.9816596	total: 4.51ms	remaining: 146ms
3:	learn: 0.9752005	total: 6.08ms	remaining: 146ms
4:	learn: 0.9685956	total: 7.43ms	remaining: 141ms
5:	learn: 0.9623726	total: 8.71ms	remaining: 137ms
6:	learn: 0.9562874	total: 10.1ms	remaining: 134ms
7:	learn: 0.9503389	total: 11.5ms	remaining: 133ms
8:	learn: 0.9445108	total: 12.9ms	remaining: 131ms
9:	learn: 0.9385901	total: 14.5ms	remaining: 130ms
10:	learn: 0.9327887	total: 16.1ms	remaining: 131ms
11:	learn: 0.9267606	total: 17.5ms	remaining: 128ms
12:	learn: 0.9207857	total: 19.1ms	remaining: 128ms
13:	learn: 0.9148653	total: 20.5ms	remaining: 126ms
14:	learn: 0.9091220	total: 21.9ms	remaining: 124ms
15:	learn: 0.9032216	total: 23.3ms	remaining: 123ms
16:	learn: 0.8977679	total: 24.8ms	remaining: 121ms
17:	learn: 0.8922738	total: 26.4ms	remaining: 120ms
18:	learn: 0.8865372	total: 28.1ms	remaining: 120ms
19:	learn: 0.8809984	t

0:	learn: 0.9468535	total: 1.59ms	remaining: 157ms
1:	learn: 0.8899311	total: 3.41ms	remaining: 167ms
2:	learn: 0.8374860	total: 5.04ms	remaining: 163ms
3:	learn: 0.7896540	total: 6.85ms	remaining: 165ms
4:	learn: 0.7479091	total: 8.64ms	remaining: 164ms
5:	learn: 0.7103541	total: 10.2ms	remaining: 160ms
6:	learn: 0.6794490	total: 12ms	remaining: 159ms
7:	learn: 0.6518620	total: 13.6ms	remaining: 156ms
8:	learn: 0.6288764	total: 15.3ms	remaining: 155ms
9:	learn: 0.6060904	total: 17.1ms	remaining: 153ms
10:	learn: 0.5873238	total: 18.6ms	remaining: 151ms
11:	learn: 0.5707852	total: 20.2ms	remaining: 148ms
12:	learn: 0.5565320	total: 21.9ms	remaining: 146ms
13:	learn: 0.5439539	total: 23.6ms	remaining: 145ms
14:	learn: 0.5322475	total: 25.2ms	remaining: 143ms
15:	learn: 0.5231965	total: 26.8ms	remaining: 141ms
16:	learn: 0.5154968	total: 28.3ms	remaining: 138ms
17:	learn: 0.5089987	total: 29.9ms	remaining: 136ms
18:	learn: 0.5028066	total: 31.6ms	remaining: 135ms
19:	learn: 0.4974325	tot

...

52:	learn: 0.3633699	total: 947ms	remaining: 840ms
53:	learn: 0.3624700	total: 965ms	remaining: 822ms
54:	learn: 0.3613277	total: 983ms	remaining: 804ms
55:	learn: 0.3600526	total: 1s	remaining: 786ms
56:	learn: 0.3587662	total: 1.02s	remaining: 768ms
57:	learn: 0.3576846	total: 1.04s	remaining: 751ms
58:	learn: 0.3562951	total: 1.05s	remaining: 733ms
59:	learn: 0.3552311	total: 1.07s	remaining: 715ms
60:	learn: 0.3537370	total: 1.09s	remaining: 696ms
61:	learn: 0.3528031	total: 1.11s	remaining: 678ms
62:	learn: 0.3522536	total: 1.12s	remaining: 660ms
63:	learn: 0.3509942	total: 1.14s	remaining: 642ms
64:	learn: 0.3499189	total: 1.16s	remaining: 625ms
65:	learn: 0.3491858	total: 1.18s	remaining: 608ms
66:	learn: 0.3480894	total: 1.2s	remaining: 589ms
67:	learn: 0.3467380	total: 1.21s	remaining: 572ms
68:	learn: 0.3456667	total: 1.23s	remaining: 553ms
69:	learn: 0.3445848	total: 1.25s	remaining: 535ms
70:	learn: 0.3439635	total: 1.27s	remaining: 517ms
71:	learn: 0.3426628	total: 1.28s	r

24:	learn: 0.3070099	total: 445ms	remaining: 1.33s
25:	learn: 0.3020646	total: 462ms	remaining: 1.31s
26:	learn: 0.2988184	total: 480ms	remaining: 1.3s
27:	learn: 0.2948475	total: 497ms	remaining: 1.28s
28:	learn: 0.2915500	total: 514ms	remaining: 1.26s
29:	learn: 0.2870844	total: 532ms	remaining: 1.24s
30:	learn: 0.2830298	total: 551ms	remaining: 1.23s
31:	learn: 0.2803734	total: 568ms	remaining: 1.21s
32:	learn: 0.2786886	total: 585ms	remaining: 1.19s
33:	learn: 0.2757327	total: 601ms	remaining: 1.17s
34:	learn: 0.2723911	total: 618ms	remaining: 1.15s
35:	learn: 0.2687461	total: 635ms	remaining: 1.13s
36:	learn: 0.2652559	total: 654ms	remaining: 1.11s
37:	learn: 0.2614687	total: 672ms	remaining: 1.1s
38:	learn: 0.2586449	total: 690ms	remaining: 1.08s
39:	learn: 0.2554981	total: 708ms	remaining: 1.06s
40:	learn: 0.2527963	total: 725ms	remaining: 1.04s
41:	learn: 0.2512905	total: 742ms	remaining: 1.02s
42:	learn: 0.2485259	total: 760ms	remaining: 1.01s
43:	learn: 0.2465884	total: 778ms

92:	learn: 0.1786724	total: 1.72s	remaining: 130ms
93:	learn: 0.1776010	total: 1.74s	remaining: 111ms
94:	learn: 0.1764505	total: 1.76s	remaining: 92.6ms
95:	learn: 0.1748331	total: 1.78s	remaining: 74.1ms
96:	learn: 0.1737765	total: 1.79s	remaining: 55.5ms
97:	learn: 0.1729991	total: 1.81s	remaining: 37ms
98:	learn: 0.1717963	total: 1.83s	remaining: 18.5ms
99:	learn: 0.1710675	total: 1.85s	remaining: 0us
0:	learn: 0.6849020	total: 18.6ms	remaining: 1.84s
1:	learn: 0.5399319	total: 35.4ms	remaining: 1.74s
2:	learn: 0.4746744	total: 53.6ms	remaining: 1.73s
3:	learn: 0.4432355	total: 71ms	remaining: 1.71s
4:	learn: 0.4220190	total: 88.9ms	remaining: 1.69s
5:	learn: 0.4093840	total: 107ms	remaining: 1.67s
6:	learn: 0.4003452	total: 125ms	remaining: 1.66s
7:	learn: 0.3894218	total: 142ms	remaining: 1.63s
8:	learn: 0.3805778	total: 160ms	remaining: 1.62s
9:	learn: 0.3737521	total: 178ms	remaining: 1.6s
10:	learn: 0.3670119	total: 195ms	remaining: 1.58s
11:	learn: 0.3617097	total: 213ms	rema

57:	learn: 0.2158112	total: 1.05s	remaining: 761ms
58:	learn: 0.2140077	total: 1.07s	remaining: 745ms
59:	learn: 0.2121296	total: 1.09s	remaining: 727ms
60:	learn: 0.2105224	total: 1.11s	remaining: 709ms
61:	learn: 0.2086543	total: 1.13s	remaining: 691ms
62:	learn: 0.2075616	total: 1.15s	remaining: 673ms
63:	learn: 0.2065639	total: 1.16s	remaining: 654ms
64:	learn: 0.2053066	total: 1.18s	remaining: 635ms
65:	learn: 0.2036226	total: 1.2s	remaining: 617ms
66:	learn: 0.2020228	total: 1.22s	remaining: 599ms
67:	learn: 0.2008214	total: 1.23s	remaining: 580ms
68:	learn: 0.1989501	total: 1.25s	remaining: 563ms
69:	learn: 0.1976059	total: 1.27s	remaining: 545ms
70:	learn: 0.1960788	total: 1.29s	remaining: 527ms
71:	learn: 0.1943072	total: 1.31s	remaining: 509ms
72:	learn: 0.1927956	total: 1.32s	remaining: 490ms
73:	learn: 0.1908345	total: 1.34s	remaining: 472ms
74:	learn: 0.1897133	total: 1.36s	remaining: 454ms
75:	learn: 0.1883781	total: 1.38s	remaining: 436ms
76:	learn: 0.1873728	total: 1.4s

39:	learn: 0.3015652	total: 190ms	remaining: 286ms
40:	learn: 0.2987715	total: 196ms	remaining: 282ms
41:	learn: 0.2970006	total: 202ms	remaining: 279ms
42:	learn: 0.2955705	total: 207ms	remaining: 275ms
43:	learn: 0.2936521	total: 213ms	remaining: 271ms
44:	learn: 0.2918901	total: 218ms	remaining: 266ms
45:	learn: 0.2890011	total: 223ms	remaining: 261ms
46:	learn: 0.2872530	total: 227ms	remaining: 257ms
47:	learn: 0.2861983	total: 232ms	remaining: 252ms
48:	learn: 0.2845249	total: 237ms	remaining: 247ms
49:	learn: 0.2826289	total: 242ms	remaining: 242ms
50:	learn: 0.2807978	total: 247ms	remaining: 237ms
51:	learn: 0.2785694	total: 252ms	remaining: 233ms
52:	learn: 0.2770186	total: 257ms	remaining: 228ms
53:	learn: 0.2756798	total: 261ms	remaining: 223ms
54:	learn: 0.2739369	total: 266ms	remaining: 218ms
55:	learn: 0.2720973	total: 271ms	remaining: 213ms
56:	learn: 0.2703519	total: 276ms	remaining: 208ms
57:	learn: 0.2688054	total: 280ms	remaining: 203ms
58:	learn: 0.2669973	total: 285

In [27]:


# Define hyperparameter grids for each model
rf_grid = {'n_estimators': [101,111,91], 
           'min_samples_split': [2,3,4], 
           'max_features': ['auto'], 
           'max_depth': [13,14,12]
 
}

hgb_grid = {'min_samples_leaf': [11,12,13], 
            'max_depth': [4,5,6,7], 
            'learning_rate': [0.6,0.4,0.5], 
            'l2_regularization':[ 10]
}

cb_grid = {'iterations': [100], 
            'depth': [8,9,7,None], 
            'learning_rate': [0.6,0.4,0.5,None], 
            'l2_leaf_reg':[ 6,7,8,None]
}


# Perform hyperparameter tuning using GridSearchCV
cb_model = CatBoostRegressor()
cb_cv = GridSearchCV(cb_model, cb_grid, cv=5, scoring="neg_mean_absolute_error")
cb_cv.fit(X_train, y_train)

rf_model = RandomForestRegressor()
rf_cv = GridSearchCV(rf_model, rf_grid, cv=5, scoring="neg_mean_absolute_error")
rf_cv.fit(X_train, y_train)

hgb_model = HistGradientBoostingRegressor()
hgb_cv = GridSearchCV(hgb_model, hgb_grid, cv=5, scoring="neg_mean_absolute_error")
hgb_cv.fit(X_train, y_train)



# Print the best hyperparameters for each model
print("Best CatBoost hyperparameters:", cb_cv.best_params_)
print("Best RandomForestRegressor hyperparameters:", rf_cv.best_params_)
print("Best HistGradientBoostingRegressor hyperparameters:", hgb_cv.best_params_)




0:	learn: 0.6305342	total: 5.57ms	remaining: 551ms
1:	learn: 0.5110500	total: 10.7ms	remaining: 523ms
2:	learn: 0.4571837	total: 15.4ms	remaining: 499ms
3:	learn: 0.4421223	total: 19.7ms	remaining: 473ms
4:	learn: 0.4271434	total: 23.8ms	remaining: 452ms
5:	learn: 0.4194576	total: 28ms	remaining: 438ms
6:	learn: 0.4095880	total: 31.9ms	remaining: 424ms
7:	learn: 0.4034834	total: 35.9ms	remaining: 413ms
8:	learn: 0.3963731	total: 39.8ms	remaining: 402ms
9:	learn: 0.3932888	total: 43.8ms	remaining: 394ms
10:	learn: 0.3903376	total: 47.5ms	remaining: 384ms
11:	learn: 0.3837164	total: 51.4ms	remaining: 377ms
12:	learn: 0.3792756	total: 55.3ms	remaining: 370ms
13:	learn: 0.3727918	total: 59.1ms	remaining: 363ms
14:	learn: 0.3664704	total: 62.9ms	remaining: 357ms
15:	learn: 0.3610056	total: 66.9ms	remaining: 351ms
16:	learn: 0.3560139	total: 70.9ms	remaining: 346ms
17:	learn: 0.3503371	total: 74.8ms	remaining: 341ms
18:	learn: 0.3447166	total: 78.6ms	remaining: 335ms
19:	learn: 0.3404946	tot

69:	learn: 0.2364985	total: 279ms	remaining: 120ms
70:	learn: 0.2343966	total: 284ms	remaining: 116ms
71:	learn: 0.2328889	total: 288ms	remaining: 112ms
72:	learn: 0.2322059	total: 293ms	remaining: 108ms
73:	learn: 0.2305626	total: 298ms	remaining: 105ms
74:	learn: 0.2291960	total: 302ms	remaining: 101ms
75:	learn: 0.2282351	total: 306ms	remaining: 96.5ms
76:	learn: 0.2271597	total: 309ms	remaining: 92.4ms
77:	learn: 0.2257626	total: 314ms	remaining: 88.6ms
78:	learn: 0.2247483	total: 318ms	remaining: 84.5ms
79:	learn: 0.2234489	total: 322ms	remaining: 80.5ms
80:	learn: 0.2222997	total: 326ms	remaining: 76.4ms
81:	learn: 0.2211177	total: 330ms	remaining: 72.3ms
82:	learn: 0.2200102	total: 333ms	remaining: 68.3ms
83:	learn: 0.2186894	total: 337ms	remaining: 64.2ms
84:	learn: 0.2177926	total: 341ms	remaining: 60.2ms
85:	learn: 0.2164685	total: 345ms	remaining: 56.2ms
86:	learn: 0.2156697	total: 349ms	remaining: 52.1ms
87:	learn: 0.2151434	total: 352ms	remaining: 48ms
88:	learn: 0.2137439

66:	learn: 0.2284525	total: 276ms	remaining: 136ms
67:	learn: 0.2271527	total: 281ms	remaining: 132ms
68:	learn: 0.2259414	total: 285ms	remaining: 128ms
69:	learn: 0.2244161	total: 291ms	remaining: 125ms
70:	learn: 0.2234762	total: 295ms	remaining: 120ms
71:	learn: 0.2230295	total: 298ms	remaining: 116ms
72:	learn: 0.2219146	total: 302ms	remaining: 112ms
73:	learn: 0.2206274	total: 307ms	remaining: 108ms
74:	learn: 0.2188413	total: 311ms	remaining: 104ms
75:	learn: 0.2180072	total: 315ms	remaining: 99.5ms
76:	learn: 0.2162045	total: 319ms	remaining: 95.3ms
77:	learn: 0.2150895	total: 323ms	remaining: 91.1ms
78:	learn: 0.2138104	total: 327ms	remaining: 86.9ms
79:	learn: 0.2127985	total: 331ms	remaining: 82.7ms
80:	learn: 0.2115862	total: 335ms	remaining: 78.6ms
81:	learn: 0.2109596	total: 339ms	remaining: 74.4ms
82:	learn: 0.2101031	total: 343ms	remaining: 70.2ms
83:	learn: 0.2089593	total: 347ms	remaining: 66ms
84:	learn: 0.2077551	total: 351ms	remaining: 61.9ms
85:	learn: 0.2075039	to

67:	learn: 0.2649065	total: 273ms	remaining: 129ms
68:	learn: 0.2634706	total: 278ms	remaining: 125ms
69:	learn: 0.2625999	total: 282ms	remaining: 121ms
70:	learn: 0.2612873	total: 286ms	remaining: 117ms
71:	learn: 0.2596464	total: 290ms	remaining: 113ms
72:	learn: 0.2581168	total: 294ms	remaining: 109ms
73:	learn: 0.2567849	total: 298ms	remaining: 105ms
74:	learn: 0.2558848	total: 302ms	remaining: 101ms
75:	learn: 0.2544524	total: 306ms	remaining: 96.7ms
76:	learn: 0.2530565	total: 310ms	remaining: 92.6ms
77:	learn: 0.2520149	total: 314ms	remaining: 88.6ms
78:	learn: 0.2510118	total: 318ms	remaining: 84.5ms
79:	learn: 0.2496412	total: 322ms	remaining: 80.5ms
80:	learn: 0.2490294	total: 326ms	remaining: 76.4ms
81:	learn: 0.2481507	total: 330ms	remaining: 72.4ms
82:	learn: 0.2470486	total: 334ms	remaining: 68.3ms
83:	learn: 0.2457010	total: 337ms	remaining: 64.3ms
84:	learn: 0.2452755	total: 341ms	remaining: 60.3ms
85:	learn: 0.2441311	total: 345ms	remaining: 56.2ms
86:	learn: 0.2431069

66:	learn: 0.2681380	total: 269ms	remaining: 132ms
67:	learn: 0.2662169	total: 273ms	remaining: 128ms
68:	learn: 0.2641261	total: 278ms	remaining: 125ms
69:	learn: 0.2622510	total: 282ms	remaining: 121ms
70:	learn: 0.2608540	total: 286ms	remaining: 117ms
71:	learn: 0.2598103	total: 290ms	remaining: 113ms
72:	learn: 0.2584572	total: 294ms	remaining: 109ms
73:	learn: 0.2572035	total: 298ms	remaining: 105ms
74:	learn: 0.2565562	total: 302ms	remaining: 101ms
75:	learn: 0.2558368	total: 306ms	remaining: 96.8ms
76:	learn: 0.2546167	total: 310ms	remaining: 92.7ms
77:	learn: 0.2536632	total: 314ms	remaining: 88.7ms
78:	learn: 0.2524855	total: 319ms	remaining: 84.7ms
79:	learn: 0.2516147	total: 323ms	remaining: 80.6ms
80:	learn: 0.2506451	total: 326ms	remaining: 76.6ms
81:	learn: 0.2492856	total: 330ms	remaining: 72.5ms
82:	learn: 0.2479181	total: 334ms	remaining: 68.4ms
83:	learn: 0.2464711	total: 338ms	remaining: 64.4ms
84:	learn: 0.2454620	total: 342ms	remaining: 60.3ms
85:	learn: 0.2443850	

...

19:	learn: 0.3812694	total: 47.1ms	remaining: 189ms
20:	learn: 0.3792088	total: 49.5ms	remaining: 186ms
21:	learn: 0.3759983	total: 51.6ms	remaining: 183ms
22:	learn: 0.3728348	total: 53.9ms	remaining: 181ms
23:	learn: 0.3693647	total: 56.3ms	remaining: 178ms
24:	learn: 0.3664861	total: 58.6ms	remaining: 176ms
25:	learn: 0.3629289	total: 60.7ms	remaining: 173ms
26:	learn: 0.3594099	total: 62.8ms	remaining: 170ms
27:	learn: 0.3566308	total: 65ms	remaining: 167ms
28:	learn: 0.3533457	total: 67.1ms	remaining: 164ms
29:	learn: 0.3503796	total: 69.3ms	remaining: 162ms
30:	learn: 0.3489670	total: 71.7ms	remaining: 160ms
31:	learn: 0.3474478	total: 73.9ms	remaining: 157ms
32:	learn: 0.3443479	total: 76.1ms	remaining: 154ms
33:	learn: 0.3420897	total: 78.4ms	remaining: 152ms
34:	learn: 0.3398855	total: 80.4ms	remaining: 149ms
35:	learn: 0.3385706	total: 82.8ms	remaining: 147ms
36:	learn: 0.3365959	total: 85.1ms	remaining: 145ms
37:	learn: 0.3354167	total: 87.3ms	remaining: 142ms
38:	learn: 0.3

1:	learn: 0.5612312	total: 5.93ms	remaining: 290ms
2:	learn: 0.5001111	total: 8.71ms	remaining: 282ms
3:	learn: 0.4575543	total: 11.4ms	remaining: 273ms
4:	learn: 0.4467158	total: 13.8ms	remaining: 263ms
5:	learn: 0.4394565	total: 16.2ms	remaining: 253ms
6:	learn: 0.4291997	total: 18.2ms	remaining: 242ms
7:	learn: 0.4246561	total: 20.3ms	remaining: 234ms
8:	learn: 0.4212007	total: 22.4ms	remaining: 227ms
9:	learn: 0.4162412	total: 24.5ms	remaining: 221ms
10:	learn: 0.4113298	total: 26.6ms	remaining: 216ms
11:	learn: 0.4077374	total: 28.8ms	remaining: 211ms
12:	learn: 0.4057324	total: 31ms	remaining: 207ms
13:	learn: 0.4020467	total: 33ms	remaining: 203ms
14:	learn: 0.4000657	total: 35.1ms	remaining: 199ms
15:	learn: 0.3973932	total: 37.2ms	remaining: 195ms
16:	learn: 0.3956249	total: 39.3ms	remaining: 192ms
17:	learn: 0.3900119	total: 41.4ms	remaining: 189ms
18:	learn: 0.3855879	total: 43.5ms	remaining: 185ms
19:	learn: 0.3828482	total: 45.7ms	remaining: 183ms
20:	learn: 0.3802875	tota

Learning rate set to 0.378126
0:	learn: 0.7608228	total: 2.57ms	remaining: 254ms
1:	learn: 0.6179346	total: 5.01ms	remaining: 246ms
2:	learn: 0.5441496	total: 7.35ms	remaining: 238ms
3:	learn: 0.4932423	total: 9.76ms	remaining: 234ms
4:	learn: 0.4718881	total: 12.1ms	remaining: 230ms
5:	learn: 0.4596067	total: 14.7ms	remaining: 230ms
6:	learn: 0.4504631	total: 17ms	remaining: 226ms
7:	learn: 0.4395935	total: 19.4ms	remaining: 224ms
8:	learn: 0.4337758	total: 21.9ms	remaining: 222ms
9:	learn: 0.4261974	total: 24.3ms	remaining: 218ms
10:	learn: 0.4223814	total: 26.5ms	remaining: 214ms
11:	learn: 0.4186827	total: 28.5ms	remaining: 209ms
12:	learn: 0.4147949	total: 30.6ms	remaining: 205ms
13:	learn: 0.4119157	total: 32.7ms	remaining: 201ms
14:	learn: 0.4081411	total: 34.9ms	remaining: 198ms
15:	learn: 0.4054384	total: 37.1ms	remaining: 195ms
16:	learn: 0.4039678	total: 39.1ms	remaining: 191ms
17:	learn: 0.4014436	total: 41.2ms	remaining: 188ms
18:	learn: 0.3998322	total: 43.2ms	remaining: 

Learning rate set to 0.378126
0:	learn: 0.7651543	total: 2.41ms	remaining: 238ms
1:	learn: 0.6128351	total: 4.7ms	remaining: 230ms
2:	learn: 0.5375416	total: 7.21ms	remaining: 233ms
3:	learn: 0.4988075	total: 9.59ms	remaining: 230ms
4:	learn: 0.4685544	total: 12ms	remaining: 228ms
5:	learn: 0.4568995	total: 14.7ms	remaining: 230ms
6:	learn: 0.4398068	total: 16.9ms	remaining: 225ms
7:	learn: 0.4340841	total: 19.3ms	remaining: 222ms
8:	learn: 0.4297385	total: 21.5ms	remaining: 217ms
9:	learn: 0.4224596	total: 23.5ms	remaining: 212ms
10:	learn: 0.4181012	total: 25.6ms	remaining: 207ms
11:	learn: 0.4138674	total: 27.7ms	remaining: 203ms
12:	learn: 0.4125209	total: 29.9ms	remaining: 200ms
13:	learn: 0.4083901	total: 32.2ms	remaining: 198ms
14:	learn: 0.4040590	total: 34.3ms	remaining: 194ms
15:	learn: 0.4006683	total: 36.4ms	remaining: 191ms
16:	learn: 0.3959009	total: 38.6ms	remaining: 188ms
17:	learn: 0.3942663	total: 40.6ms	remaining: 185ms
18:	learn: 0.3902574	total: 42.7ms	remaining: 1

0:	learn: 0.7413608	total: 5.49ms	remaining: 544ms
1:	learn: 0.5995983	total: 11ms	remaining: 537ms
2:	learn: 0.5109714	total: 15.9ms	remaining: 514ms
3:	learn: 0.4718465	total: 21.1ms	remaining: 505ms
4:	learn: 0.4442305	total: 26.4ms	remaining: 501ms
5:	learn: 0.4285960	total: 31.6ms	remaining: 494ms
6:	learn: 0.4184373	total: 36.6ms	remaining: 486ms
7:	learn: 0.4136389	total: 41.8ms	remaining: 480ms
8:	learn: 0.4080184	total: 47.2ms	remaining: 477ms
9:	learn: 0.4046178	total: 52ms	remaining: 468ms
10:	learn: 0.3985713	total: 57ms	remaining: 461ms
11:	learn: 0.3953481	total: 61.7ms	remaining: 452ms
12:	learn: 0.3923593	total: 66.3ms	remaining: 443ms
13:	learn: 0.3869651	total: 71ms	remaining: 436ms
14:	learn: 0.3827511	total: 75.6ms	remaining: 429ms
15:	learn: 0.3791999	total: 80.6ms	remaining: 423ms
16:	learn: 0.3771482	total: 85.5ms	remaining: 418ms
17:	learn: 0.3737423	total: 90.2ms	remaining: 411ms
18:	learn: 0.3709044	total: 95ms	remaining: 405ms
19:	learn: 0.3669387	total: 99.8

In [28]:

# Evaluate the best models on the test set
rf_best = rf_cv.best_estimator_
rf_mae_test = mean_absolute_error(y_test, rf_best.predict(X_test))
rf_mae_train = mean_absolute_error(y_train, rf_best.predict(X_train))

print("RandomForestRegressor MAE on test set:{} and train set {}".format(rf_mae_test,rf_mae_train))

hgb_best = hgb_cv.best_estimator_
hgb_mae_test = mean_absolute_error(y_test, hgb_best.predict(X_test))
hgb_mae_train = mean_absolute_error(y_train, hgb_best.predict(X_train))

print("HistGradientBoostingRegressor MAE on test set:{} and train set {}".format(hgb_mae_test,hgb_mae_train))


cb_best = cb_cv.best_estimator_
cb_mae_test = mean_absolute_error(y_test, cb_best.predict(X_test))
cb_mae_train = mean_absolute_error(y_train, cb_best.predict(X_train))

print("CatBoost MAE on test set:{} and train set {}".format(cb_mae_test,cb_mae_train))

RandomForestRegressor MAE on test set:0.256344084807589 and train set 0.15113085980339191
HistGradientBoostingRegressor MAE on test set:0.23304730284843 and train set 0.1725730683317913
CatBoost MAE on test set:0.2262503778072128 and train set 0.16375325006413385
